In [0]:
CREATE TABLE IF NOT EXISTS gold.dim_items (
  item_sk BIGINT GENERATED ALWAYS AS IDENTITY,
  order_id STRING,
  order_item_id INT,
  product_id STRING,
  seller_id STRING,
  shipping_limit_date TIMESTAMP,
  price DECIMAL(10,2),
  freight_value DECIMAL(10,2)
) USING DELTA

  

In [0]:
%python
items_df = spark.read.table("silver.order_items_cleaned")
items_df.createOrReplaceTempView("items_view")

In [0]:
MERGE INTO gold.dim_items AS t
USING items_view AS s
ON t.order_id = s.order_id

WHEN MATCHED THEN
  UPDATE SET 
  t.order_item_id = s.order_item_id,
  t.product_id = s.product_id,
  t.price = s.price,
  t.freight_value = s.freight_value

WHEN NOT MATCHED
  THEN INSERT (
    order_id,
    order_item_id,
    product_id,
    seller_id,
    shipping_limit_date,
    price,
    freight_value
  ) VALUES (
    s.order_id,
    s.order_item_id,
    s.product_id,
    s.seller_id,
    s.shipping_limit_date,
    s.price,
    s.freight_value
  )

In [0]:
SELECT * FROM gold.dim_items LIMIT 10;